## Imports

In [8]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

## Read data from CSV

In [9]:
df = pd.read_csv(r'https://raw.githubusercontent.com/Jordwy/Data-Mining/refs/heads/main/Applied%20Practice%205%20-%20Data.csv')

## Seperate Data into Features and Class labels 

In [15]:
def pre_process(df: pd.DataFrame):

    # Seperate class from features
    features = df.drop('edible', axis=1)
    class_labels = df['edible']

    # Shuffle data
    features = features.sample(frac=1, random_state=1234)
    class_labels = class_labels.sample(frac=1, random_state=1234)

    # Seperate into test and train
    n = int(features.shape[0] * 0.8) # get 80% index num
    train_features, train_class_labels  = features.iloc[:n], class_labels.iloc[:n]
    test_features,  test_class_labels   = features.iloc[n:], class_labels.iloc[n:]

    return train_features, train_class_labels, test_features, test_class_labels




# Task 3 - Training a decision tree with Scikit-learn

**Exercise 1**: Create a function that takes dataframe as an input argument and converts all the
columns with “object” datatype, into categorical series with appropriate codes. 

*Hint: To convert a column of “object” dtype with string values to integer values you need to first convert into an accessor object for categorical properties of the pandas Series values and then you can use the series category attributes such as “codes”.*


## Exercize 1

In [11]:
def col_convert(df:pd.DataFrame) -> pd.DataFrame:
    '''Convert all columns of dtype 'object' into catagory codes (int)'''
    
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype('category').cat.codes

    return df

In [19]:
df = col_convert(df)
train_features, train_class_labels, test_features, test_class_labels = pre_process(df)


In [21]:
classifier = DecisionTreeClassifier()

In [ ]:
classifier.fit(trai)